In [1]:
# Load data
import pandas as pd
from pathlib import Path

# Load data
df = pd.read_csv('../data/processed/checkin_data_with_sentiment.csv')

print("="*70)
print("LOOKER STUDIO DATA PREPARATION")
print("="*70)
print(f"\nDataset loaded: {len(df)} rows")

LOOKER STUDIO DATA PREPARATION

Dataset loaded: 372 rows


In [2]:
# dashboard overview
summary_metrics = pd.DataFrame({
    'Metric': [
        'Total Responses',
        'Win Responses',
        'Loss Responses',
        'Blocker Responses',
        'Avg Win Sentiment',
        'Avg Loss Sentiment',
        'Avg Blocker Sentiment'
    ],
    'Value': [
        len(df),
        df['Win'].notna().sum(),
        df['Loss'].notna().sum(),
        df['Blocker'].notna().sum(),
        round(df['Win_sentiment'].mean(), 3),
        round(df['Loss_sentiment'].mean(), 3),
        round(df['Blocker_sentiment'].mean(), 3)
    ],
    'Category': [
        'Overview',
        'Overview',
        'Overview',
        'Overview',
        'Sentiment',
        'Sentiment',
        'Sentiment'
    ]
})

print("\nSummary Metrics for Dashboard:")
display(summary_metrics)

# Export
summary_metrics.to_csv('../reports/looker_summary_metrics.csv', index=False)
print("Saved: ../reports/looker_summary_metrics.csv")


Summary Metrics for Dashboard:


,Metric,Value,Category
0,Total Responses,372.000,Overview
1,Win Responses,360.000,Overview
2,Loss Responses,303.000,Overview
3,Blocker Responses,277.000,Overview
4,Avg Win Sentiment,0.214,Sentiment
5,Avg Loss Sentiment,0.035,Sentiment
6,Avg Blocker Sentiment,-0.006,Sentiment


Saved: ../reports/looker_summary_metrics.csv


In [3]:
# Time series analysis
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Week'] = df['Timestamp'].dt.isocalendar().week
df['Month'] = df['Timestamp'].dt.month_name()

# Weekly response counts
weekly_responses = df.groupby('Week').agg({
    'Win': 'count',
    'Loss': lambda x: x.notna().sum(),
    'Blocker': lambda x: x.notna().sum()
}).reset_index()

weekly_responses.columns = ['Week', 'Total_Responses', 'Loss_Count', 'Blocker_Count']

print("\nWeekly Response Trends:")
display(weekly_responses)

# Export
weekly_responses.to_csv('../reports/looker_weekly_trends.csv', index=False)
print(" Saved: ../reports/looker_weekly_trends.csv")


Weekly Response Trends:


,Week,Total_Responses,Loss_Count,Blocker_Count
0,28,32,24,29
1,29,34,24,24
2,30,5,5,2
3,31,3,3,1
4,32,14,8,6
5,34,34,27,25
6,35,27,20,16
7,36,8,6,7
8,37,12,11,10
9,38,4,3,3


 Saved: ../reports/looker_weekly_trends.csv


In [ ]:
import os

theme_files = [
    '../reports/top_win_themes.csv',
    '../reports/top_loss_themes.csv',
    '../reports/top_blocker_themes.csv'
]

print("\nTheme Files for Looker:")
for f in theme_files:
    if os.path.exists(f):
        print(f"✅ {f}")
    else:
        print(f"❌ {f} - MISSING!")


Theme Files for Looker:
✅ ../reports/top_win_themes.csv
✅ ../reports/top_loss_themes.csv
✅ ../reports/top_blocker_themes.csv


In [6]:
def categorize_sentiment(score):
    if pd.isna(score):
        return 'No Response'
    elif score >= 0.3:
        return 'Positive'
    elif score >= -0.1:
        return 'Neutral'
    else:
        return 'Negative'

df['Win_sentiment_category'] = df['Win_sentiment'].apply(categorize_sentiment)
df['Loss_sentiment_category'] = df['Loss_sentiment'].apply(categorize_sentiment)
df['Blocker_sentiment_category'] = df['Blocker_sentiment'].apply(categorize_sentiment)

sentiment_breakdown = pd.DataFrame({
    'Category': ['Win', 'Loss', 'Blocker'],
    'Positive': [
        (df['Win_sentiment_category'] == 'Positive').sum(),
        (df['Loss_sentiment_category'] == 'Positive').sum(),
        (df['Blocker_sentiment_category'] == 'Positive').sum()
    ],
    'Neutral': [
        (df['Win_sentiment_category'] == 'Neutral').sum(),
        (df['Loss_sentiment_category'] == 'Neutral').sum(),
        (df['Blocker_sentiment_category'] == 'Neutral').sum()
    ],
    'Negative': [
        (df['Win_sentiment_category'] == 'Negative').sum(),
        (df['Loss_sentiment_category'] == 'Negative').sum(),
        (df['Blocker_sentiment_category'] == 'Negative').sum()
    ]
})

sentiment_breakdown
sentiment_breakdown.to_csv('../reports/looker_sentiment_breakdown.csv', index=False)
print(" Saved: ../reports/looker_sentiment_breakdown.csv")


 Saved: ../reports/looker_sentiment_breakdown.csv


In [7]:
# Export master dataset for Looker Studio
looker_export = df[[
    'Timestamp',
    'Win', 'Loss', 'Blocker',
    'Win_sentiment', 'Loss_sentiment', 'Blocker_sentiment',
    'Win_sentiment_category', 'Loss_sentiment_category', 'Blocker_sentiment_category',
    'Week', 'Month'
]].copy()

# Export
looker_export.to_csv('../reports/looker_master_data.csv', index=False)
print("\n --> Master dataset exported for Looker Studio!")
print(f"   File: ../reports/looker_master_data.csv")
print(f"   Rows: {len(looker_export)}")
print(f"   Columns: {len(looker_export.columns)}")

print("\n" + "="*70)
print("LOOKER STUDIO FILES READY FOR UPLOAD")
print("="*70)
print("\nFiles created:")
print("1. looker_summary_metrics.csv - Overview metrics")
print("2. looker_weekly_trends.csv - Time series data")
print("3. looker_sentiment_breakdown.csv - Sentiment categories")
print("4. looker_master_data.csv - Complete dataset")
print("5. top_win_themes.csv - Win themes")
print("6. top_loss_themes.csv - Loss themes")
print("7. top_blocker_themes.csv - Blocker themes")


 --> Master dataset exported for Looker Studio!
   File: ../reports/looker_master_data.csv
   Rows: 372
   Columns: 12

LOOKER STUDIO FILES READY FOR UPLOAD

Files created:
1. looker_summary_metrics.csv - Overview metrics
2. looker_weekly_trends.csv - Time series data
3. looker_sentiment_breakdown.csv - Sentiment categories
4. looker_master_data.csv - Complete dataset
5. top_win_themes.csv - Win themes
6. top_loss_themes.csv - Loss themes
7. top_blocker_themes.csv - Blocker themes
